In [22]:
import zipfile

zip_path = "D:/PROGRAMMING LANGUAGES/PYTHON/A new project/Pneumonia Detection Dataset.zip"
extract_path = "D:/PROGRAMMING LANGUAGES/PYTHON/A new project/content/"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Extraction complete!")


Extraction complete!


In [7]:
!pip install tensorflow keras scipy glob2

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached numpy-1.23.5-cp311-cp311-win_amd64.whl.metadata (2.3 kB)
Using cached numpy-1.23.5-cp311-cp311-win_amd64.whl (14.6 MB)
  Created wheel for glob2: filename=glob2-0.7-py2.py3-none-any.whl size=9377 sha256=d4082a38cf659e446df44ad8656dede3a1ecae2ed8a3fb61d18ef3244d718b36
  Stored in directory: c:\users\varsh\appdata\local\pip\cache\wheels\96\41\06\9f8fddc6eb1d75bde63db7f491311a4ae26905212617e06eb2
Successfully built glob2

  Attempting uninstall: protobuf

   ------------- -------------------------- 1/3 [protobuf]
    Found existing installation: protobuf 5.29.4
   ------------- -------------------------- 1/3 [protobuf]
   ------------- -------------------------- 1/3 [protobuf]
    Uninstalling protobuf-5.29.4:
   ------------- -------------------------- 1/3 [protobuf]
      Successfully uninstalled protobuf-5.29.4
   ------------- -------------------------- 1/3 [protobuf]
 

  DEPRECATION: Building 'glob2' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'glob2'. Discussion can be found at https://github.com/pypa/pip/issues/6334
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
faiss-cpu 1.11.0 requires numpy<3.0,>=1.25.0, but you have numpy 1.23.5 which is incompatible.
grpcio-status 1.71.0 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 4.25.8 which is incompatible.
langchain-chroma 0.2.4 requires numpy>=1.26.0; python_version < "3.13", but you have numpy 1.23.5 which is incompatible.
langchain-community 0.3.26 requires numpy>=1.26

In [23]:
from keras.models import Model
from keras.layers import Flatten,Dense
from keras.applications.vgg16 import VGG16
import matplotlib.pyplot as plot
from glob import glob

In [24]:
IMAGESHAPE = [224, 224, 3]
training_data = 'chest_xray/train'
testing_data = 'chest_xray/test'

In [25]:
vgg_model = VGG16(input_shape=IMAGESHAPE, weights='imagenet', include_top=False)

In [26]:
for each_layer in vgg_model.layers:
	each_layer.trainable = False

In [27]:
classes = glob('chest_xray/train/*')

In [28]:
flatten_layer = Flatten()(vgg_model.output)
prediction = Dense(len(classes), activation='softmax')(flatten_layer)

In [29]:
final_model = Model(inputs=vgg_model.input, outputs=prediction)
final_model.summary()


Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

In [30]:
final_model.compile(
loss='categorical_crossentropy',
optimizer='adam',
metrics=['accuracy']
)


In [31]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
								shear_range = 0.2,
								zoom_range = 0.2,
								horizontal_flip = True)
testing_datagen = ImageDataGenerator(rescale =1. / 255)

In [32]:
import os
test_dir = "/content/chest_xray/test"

if os.path.exists(test_dir):
    print("Test folder found!")
    print("Contents:", os.listdir(test_dir))
else:
    print("Test folder not found. Extract the ZIP file again.")


Test folder not found. Extract the ZIP file again.


In [ ]:
training_set = train_datagen.flow_from_directory(
    'D:/PROGRAMMING LANGUAGES/PYTHON/A new project/content/chest_xray/train',
    target_size=(224, 224),
    batch_size=4,
    class_mode='binary'
)


Found 5232 images belonging to 2 classes.


In [40]:
test_set = testing_datagen.flow_from_directory('D:/PROGRAMMING LANGUAGES/PYTHON/A new project/content/chest_xray/test',
											target_size = (224, 224),
											batch_size = 4,
											class_mode = 'binary')


Found 624 images belonging to 2 classes.


In [42]:
x = Flatten()(vgg_model.output)
prediction = Dense(1, activation='sigmoid')(x)

final_model = Model(inputs=vgg_model.input, outputs=prediction)

final_model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

In [43]:
fitted_model = final_model.fit(
    training_set,
    validation_data=test_set,
    epochs=5,
    steps_per_epoch=training_set.samples // training_set.batch_size,
    validation_steps=test_set.samples // test_set.batch_size
)


Epoch 1/5
1308/1308 [==============================] - 2019s 2s/step - loss: 0.2044 - accuracy: 0.9283 - val_loss: 0.6601 - val_accuracy: 0.8317
Epoch 2/5
1308/1308 [==============================] - 1945s 1s/step - loss: 0.1545 - accuracy: 0.9494 - val_loss: 0.4676 - val_accuracy: 0.8510
Epoch 3/5
1308/1308 [==============================] - 2141s 2s/step - loss: 0.1305 - accuracy: 0.9574 - val_loss: 1.0571 - val_accuracy: 0.7885
Epoch 4/5
1308/1308 [==============================] - 31351s 24s/step - loss: 0.1159 - accuracy: 0.9639 - val_loss: 0.2693 - val_accuracy: 0.9327
Epoch 5/5
1308/1308 [==============================] - 1967s 2s/step - loss: 0.1207 - accuracy: 0.9644 - val_loss: 0.3550 - val_accuracy: 0.9247


In [44]:
final_model.save('our_model.h5')

In [71]:
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.vgg16 import preprocess_input
import numpy as np

model = load_model('our_model.h5')

img = image.load_img(
    'D:/PROGRAMMING LANGUAGES/PYTHON/A new project/content/chest_xray/test/NORMAL/NORMAL2-IM-0012-0001.jpeg',
    target_size=(224,224)
)

imagee = image.img_to_array(img)
imagee = np.expand_dims(imagee, axis=0)
img_data = preprocess_input(imagee)

prediction = model.predict(img_data)

if prediction[0][0] < 0.5:
    print('Person is safe.')
else:
    print('Person is affected with Pneumonia.')

print(f'Predictions: {prediction}')


1/1 [==============================] - 0s 444ms/step
Person is safe.
Predictions: [[0.]]


In [73]:
confidence = prediction[0][0] * 100

print(f"Confidence: {confidence:.2f}%")

if confidence > 70:
    print("High chance of Pneumonia")
elif confidence > 50:
    print("Moderate risk - consult doctor")
else:
    print("Normal")

Confidence: 0.00%
Normal
